In [1]:
import transformers
from transformers import (
    MT5ForConditionalGeneration,
    Seq2SeqTrainer, MT5Tokenizer, MT5Config
)

import datasets
import pandas as pd
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
import gc
import datasets
import os
import torch

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
#os.environ["WANDB_DISABLED"] = "true"
#!export CUDA_VISIBLE_DEVICES=0
#device, use_gpu = ("cuda:0", True) if torch.cuda.is_available() else ("cpu", False)
#device, use_gpu = ("cpu", False)  # Force CPU usage
device = torch.device("cpu")

C:\Users\tonnu\anaconda3\envs\transform\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !pip install datasets
# !pip install --upgrade accelerate

In [3]:
import json
checkpoint = "VietAI/vit5-base"
model = MT5ForConditionalGeneration.from_pretrained(checkpoint)
model = model.to(device)
print('load model done')
tokenizer = MT5Tokenizer.from_pretrained(checkpoint)
print('load tokenizer done')

You are using a model of type t5 to instantiate a model of type mt5. This is not supported for all configurations of models and can yield errors.


load model done
load tokenizer done


In [4]:
tdata = pd.read_excel("DATA_chuyen_cau.xlsx")
tdata = tdata.applymap(str)
tdata = tdata.reset_index()
dataset = datasets.Dataset.from_pandas(tdata)

train = dataset.train_test_split(
    train_size=0.8, test_size=0.2, shuffle = False
)

train_data = train['train']
test_data = train['test']

train_data

Dataset({
    features: ['index', 'Original', 'Paraphrase'],
    num_rows: 55
})

In [5]:
def format_dataset(example):
     return {'input': example['Original'], 'target': example['Paraphrase']}
train_data = train_data.map(format_dataset, remove_columns=train_data.column_names)
test_data = test_data.map(format_dataset, remove_columns=test_data.column_names)

def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], pad_to_max_length=True, max_length=128)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], pad_to_max_length=True, max_length=128)
    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings
train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
test_data = test_data.map(convert_to_features, batched=True, remove_columns=test_data.column_names)

Map:   0%|                                                                               | 0/55 [00:00<?, ? examples/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\tonnu\anaconda3\envs\transform\lib\site-packages\transformers\tokenization_utils_base.py:2304: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model 

In [6]:
# !pip install nltk rouge_score

In [7]:
from datasets import load_metric
rouge = load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

C:\Users\tonnu\AppData\Local\Temp\ipykernel_23232\3190919124.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
C:\Users\tonnu\anaconda3\envs\transform\lib\site-packages\datasets\load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [8]:
import torch

# Set device to CPU
device = torch.device("cpu")

# Set CUDA_VISIBLE_DEVICES environment variable to use CPU only
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

torch.cuda.empty_cache()

In [28]:
num_train_epochs = 5

data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)
training_args = Seq2SeqTrainingArguments(
    output_dir="viT5-base-1",
    per_device_train_batch_size=1,
    num_train_epochs=num_train_epochs,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    evaluation_strategy="steps",
    do_train=True,
    do_eval=True,
    logging_steps=22859,
    save_strategy="steps",
    save_steps=45718,
    eval_steps=22859,
    overwrite_output_dir=True,
    save_total_limit=4,
    load_best_model_at_end=True,
    report_to=None,
    group_by_length=True,
    #fp16=True, 
)
trainer = Seq2SeqTrainer(
    model=model,
    data_collator = data_collator,
    tokenizer = tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
trainer.train()

***** Running training *****
  Num examples = 55
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 275
  Number of trainable parameters = 253672704


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=275, training_loss=0.40245508367365057, metrics={'train_runtime': 1399.6033, 'train_samples_per_second': 0.196, 'train_steps_per_second': 0.196, 'total_flos': 47720683929600.0, 'train_loss': 0.40245508367365057, 'epoch': 5.0})

In [30]:
# run
def paraphase(text):
    inputs = tokenizer(text, padding='longest', max_length=64, return_tensors='pt')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=64)
    return tokenizer.decode(output[0], skip_special_tokens=True)

texts = [
        "Một cửa hàng có mã cửa hàng để phân biệt với các cửa hàng khác, có tên cửa hàng, địa chỉ, tên người quản lý.",
        "Một quyển sách có các thông tin gồm mã sách, tên sách, tên tác giả."
        ]
for text in texts:
    print(f'Input: {text}')
    print(f'Output: {paraphase(text)}')
    print('-'*100)

Input: Một cửa hàng có mã cửa hàng để phân biệt với các cửa hàng khác, có tên cửa hàng, địa chỉ, tên người quản lý.
Output: Mỗi cửa hàng được xác định bởi duy nhất một mã cửa hàng, tên cửa hàng, địa chỉ, tên người quản lý.
----------------------------------------------------------------------------------------------------
Input: Một quyển sách có các thông tin gồm mã sách, tên sách, tên tác giả.
Output: Mỗi quyển sách có nhiều thông tin.
----------------------------------------------------------------------------------------------------


In [31]:
def paraphase_5_(text):
    inputs = tokenizer(text, padding='longest', max_length=64, return_tensors='pt')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=64,num_beams=5, num_return_sequences=5)
    for i, output in enumerate(outputs):
        print(f'Output {i}: {tokenizer.decode(output, skip_special_tokens=True)}')
    # return tokenizer.decode(output[0], skip_special_tokens=True)

texts = [
        "Một cửa hàng có mã cửa hàng để phân biệt với các cửa hàng khác, có tên cửa hàng, địa chỉ, tên người quản lý.",
        "Một quyển sách có các thông tin gồm mã sách, tên sách, tên tác giả."
        ]
for text in texts:
    print(f'Input: {text}')
    paraphase_5_(text)
    print('-'*100)

Input: Một cửa hàng có mã cửa hàng để phân biệt với các cửa hàng khác, có tên cửa hàng, địa chỉ, tên người quản lý.
Output 0: Mỗi cửa hàng được xác định bởi duy nhất một mã cửa hàng, tên cửa hàng, địa chỉ, địa chỉ, tên người quản lý.
Output 1: Mỗi cửa hàng được xác định bởi duy nhất một mã cửa hàng, tên cửa hàng, địa chỉ, tên người quản lý.
Output 2: Mỗi cửa hàng được xác định bởi duy nhất một mã cửa hàng, tên cửa hàng, địa chỉ, số người quản lý.
Output 3: Mỗi cửa hàng được xác định bởi duy nhất một mã cửa hàng, tên cửa hàng, địa chỉ, số cửa hàng.
Output 4: Mỗi cửa hàng có nhiều cửa hàng.
----------------------------------------------------------------------------------------------------
Input: Một quyển sách có các thông tin gồm mã sách, tên sách, tên tác giả.
Output 0: Một quyển sách có nhiều thông tin. Mỗi thông tin được xác định bởi duy nhất một mã sách, tên sách, tên tác giả.
Output 1: Một quyển sách có nhiều thông tin. Mỗi thông tin có nhiều sách.
Output 2: Một quyển sách có nhiề